<a href="https://colab.research.google.com/github/hucarlos08/Geo-GEE/blob/main/PixelTransforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import folium
from folium import plugins
import geemap


# Authenticate and initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project="ee-cimat")


## Folium

# Enhanced Vegetation Index (EVI): Theory and Application

## Introduction to Vegetation Indices in Remote Sensing

The Enhanced Vegetation Index (EVI) represents a significant advancement in remote sensing technology for vegetation monitoring. Developed to overcome limitations of the Normalized Difference Vegetation Index (NDVI), EVI offers improved sensitivity in high biomass regions and greater robustness against atmospheric and soil background effects.

## Comparative Advantages of EVI over NDVI

1. **Reduced Saturation in Dense Vegetation:** While NDVI tends to saturate in areas with high leaf area index, EVI maintains sensitivity in dense forest canopies, allowing for more accurate biomass estimation—a critical parameter in environmental modeling.

2. **Atmospheric Resistance:** EVI incorporates the blue band to correct for atmospheric aerosol scattering, reducing the influence of atmospheric conditions on vegetation measurements.

3. **Soil Background Adjustment:** The algorithm includes coefficients that minimize soil brightness influences, producing more reliable results in areas with varying soil exposure.

## Mathematical Formulation

The EVI is calculated using the following equation:

$$
\text{EVI} = G \times \frac{\text{NIR} - \text{Red}}{\text{NIR} + C_1 \times \text{Red} - C_2 \times \text{Blue} + L}
$$

Where:
- NIR = Near-infrared reflectance
- Red = Red band reflectance
- Blue = Blue band reflectance
- L = Canopy background adjustment factor (typically 1)
- C₁, C₂ = Coefficients for atmospheric correction (typically 6 and 7.5, respectively)
- G = Gain factor (typically 2.5)

## Implementation with Sentinel-2 Data

For Sentinel-2 multispectral imagery, the bands correspond as follows:
- B8 (842 nm) = NIR
- B4 (665 nm) = Red
- B2 (490 nm) = Blue

The implementation formula becomes:

$$
\text{EVI} = 2.5 \times \frac{B_{8, \text{scaled}} - B_{4, \text{scaled}}}{B_{8, \text{scaled}} + 6 \times B_{4, \text{scaled}} - 7.5 \times B_{2, \text{scaled}} + 1}
$$

## Critical Preprocessing: Reflectance Scaling

When working with Sentinel-2 surface reflectance products (e.g., 'COPERNICUS/S2_SR_HARMONIZED'), values are typically scaled by a factor of 10,000. Proper normalization is essential before applying the EVI algorithm:

```python
B8_scaled = B8_original / 10000
B4_scaled = B4_original / 10000
B2_scaled = B2_original / 10000
```

Failure to normalize will result in significant calculation errors—an important consideration when implementing optimization algorithms.

## Interpretation of EVI Values

The resulting EVI values typically range from -0.2 to 1, where:
- 0.6-0.9: Dense, healthy vegetation
- 0.2-0.5: Moderate vegetation cover
- <0.2: Sparse vegetation or non-vegetated surfaces
- Negative values: Typically water bodies, snow, or clouds

In [2]:
# -----------------------------------------------------------------------------
# Notebook: Single Location EVI Calculation with GEE and geemap
# -----------------------------------------------------------------------------
# Purpose: Illustrate EVI calculation for a user-selected location.
# Designed for IRAP program, University of Texas.
# Code should be well-documented but not overly complex.
# -----------------------------------------------------------------------------

# Import necessary libraries
import ee
import geemap
from IPython.display import display # To display the map in the notebook


# --- 2. Define Locations of Interest ---
# Structure: key: {'name': 'Display Name', 'coords': [latitude, longitude]}
locations = {
    'sfo': {
        'name': 'San Francisco Bay Area, USA',
        'coords': [37.6194, -122.3774]  # lat, lon
    },
    'cimat': {
        'name': 'CIMAT, Guanajuato, MX',
        'coords': [21.0422, -101.2610]  # lat, lon
    },
    'pcty': {
        'name': 'Parque Científico y Tecnológico de Yucatán, MX',
        'coords': [21.131030340951618, -89.78090162883561] # lat, lon
    },
    'rim_fire': {
         'name': 'Rim Fire Area, California, USA',
         'coords': [37.850, -120.083]  # lat, lon for Rim Fire
     }
}

# --- 3. Select a Location to Process ---
# MODIFY THIS LINE TO CHOOSE A DIFFERENT LOCATION
# Available keys: 'sfo', 'cimat', 'pcty'
selected_location_key = 'pcty'
# -----------------------------------------

# Get the data for the selected location
if selected_location_key not in locations:
    raise ValueError(f"Error: The key '{selected_location_key}' is not defined in the 'locations' dictionary. "
                     f"Available keys are: {list(locations.keys())}")

selected_location_data = locations[selected_location_key]
location_name = selected_location_data['name']
# Coordinates are [latitude, longitude]
location_coords_lat_lon = selected_location_data['coords']
# Earth Engine's ee.Geometry.Point requires (longitude, latitude)
point_coords_lon_lat = [location_coords_lat_lon[1], location_coords_lat_lon[0]]

print(f"Processing location: {location_name} ({selected_location_key})")

# --- 4. Define Point of Interest (POI) for Earth Engine ---
poi = ee.Geometry.Point(point_coords_lon_lat) # lon, lat

# --- 5. Load Sentinel-2 Imagery ---
# We use 'COPERNICUS/S2_SR_HARMONIZED' for Surface Reflectance ( atmospherically corrected)
# which is generally preferred over 'COPERNICUS/S2' (Top of Atmosphere).
# Filter by date and location, then sort by cloud cover and take the least cloudy.
image_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterBounds(poi)
                    .filterDate('2020-06-01', '2020-07-01') # Define your date range
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) # Filter by cloud cover
                    .sort('CLOUDY_PIXEL_PERCENTAGE'))     # Sort by least cloudy

# Check if any images were found
count = image_collection.size().getInfo()
if count == 0:
    raise Exception(f"No Sentinel-2 images found for {location_name} in the specified date range and location.")

# Get the first (least cloudy) image from the filtered collection
selected_image = image_collection.first()

# --- 6. Prepare Bands for EVI Calculation ---
# Sentinel-2 bands are scaled. We need to divide by 10000 to get reflectance values.
# NIR: Band 8 (Near Infrared)
# RED: Band 4 (Red)
# BLUE: Band 2 (Blue)
nir_scaled = selected_image.select('B8').divide(10000.0)
red_scaled = selected_image.select('B4').divide(10000.0)
blue_scaled = selected_image.select('B2').divide(10000.0)

# --- 7. Compute EVI (Enhanced Vegetation Index) ---
# EVI = 2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))
# Using ee.Image.expression for the calculation.
evi_expression = selected_image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1.0))', {
        'NIR': nir_scaled,
        'RED': red_scaled,
        'BLUE': blue_scaled
    }).rename('EVI') # Rename the resulting band to 'EVI'

# --- 8. Define Visualization Parameters ---
# For EVI: typically ranges from -0.2 (barren) to +0.8 (dense vegetation)
evi_vis_params = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['#FF0000', '#FFFFFF', '#00FF00'] # Red (low), White (mid), Green (high)
}

# For Sentinel-2 True Color (RGB) visualization
s2_vis_params = {
    'bands': ['B4', 'B3', 'B2'], # Red, Green, Blue bands
    'min': 0,
    'max': 3000, # Unscaled values, adjust for brightness
    'gamma': 1.2 # Adjust gamma for contrast
}

# --- 9. Create and Display the Map using geemap ---
# Create a geemap Map object.
# Center it on the selected location's coordinates ([lat, lon])
# geemap.Map by default uses Google Maps basemap.
map_display = geemap.Map(center=location_coords_lat_lon, zoom=11)
map_display.add_basemap('HYBRID')

# Add the Sentinel-2 True Color layer for context
map_display.addLayer(
    selected_image,
    s2_vis_params,
    f'Sentinel-2 True Color - {location_name}'
)

# Add the EVI layer to the map
map_display.addLayer(
    evi_expression,
    evi_vis_params,
    f'EVI - {location_name}'
)


# Add a layer control to toggle layers on/off
map_display.addLayerControl()

# Add a marker at the point of interest
# 'location' argument for add_marker expects [lat, lon]
map_display.add_marker(
    location=location_coords_lat_lon,
    name=location_name,
)

# Display the map in the notebook output
print(f"\nDisplaying map for {location_name}...")
display(map_display)

print("\n--- Script execution finished ---")

Processing location: Parque Científico y Tecnológico de Yucatán, MX (pcty)

Displaying map for Parque Científico y Tecnológico de Yucatán, MX...


Map(center=[21.131030340951618, -89.78090162883561], controls=(WidgetControl(options=['position', 'transparent…


--- Script execution finished ---


# 🔥 Burned Area Index (BAI)

## What is BAI and Why is it Important? 🔎

Imagine a wildfire has just swept through an area. How can we quickly see from space exactly where the fire burned? That's where the **Burned Area Index (BAI)** comes in handy!

The BAI was developed by scientists (Chuvieco et al., 2002, based on ideas from Martín, 1998) to help us pinpoint burned regions using satellite images. It's like a special pair of glasses that makes burned areas stand out.

**How does it work?** Satellites see different "colors" or bands of light. After a fire:
*   **🌿 Healthy Plants:** Reflect a lot of Near-Infrared (NIR) light (invisible to us) and absorb most Red light (for photosynthesis).
*   **⚫️ Burned Areas:** Vegetation is gone or charred. This char and ash reflect less NIR light and a bit more Red light compared to healthy plants.

BAI looks at these differences. It calculates how "spectrally close" each tiny spot (pixel) in the image is to a "signature" of burnt material (like charcoal and ash) in the Red and NIR light bands.
*   **High BAI values:** Mean the pixel looks very much like burnt material. 🔥
*   **Low BAI values:** Mean the pixel looks more like unburnt plants or soil. 🌱

## The Magic Formula for BAI 🧙‍♂️

The mathematical recipe for the Burned Area Index (BAI) is:

$$ \text{BAI} = \frac{1}{(0.1 - \text{Red}_{\text{scaled}})^2 + (0.06 - \text{NIR}_{\text{scaled}})^2} $$

Let's break this down:

*   **$\text{Red}_{\text{scaled}}$**: This is the surface reflectance value (how much light is bouncing off) in the **Red** part of the light spectrum. We need this value to be a decimal between 0 and 1 (e.g., 0.2). ❤️
*   **$\text{NIR}_{\text{scaled}}$**: This is the surface reflectance value in the **Near-Infrared** (NIR) part of the light spectrum. Again, this should be a decimal between 0 and 1 (e.g., 0.05). ✨
*   **$0.1$**: This is a constant reference value for Red light. It represents a typical Red reflectance for burned material.
*   **$0.06$**: This is a constant reference value for NIR light. It represents a typical NIR reflectance for burned material.

**Think of it like this:**

The bottom part of the fraction (the denominator) is calculating something like the *distance* between:
1.  The Red and NIR values of a pixel in our satellite image.
2.  The "ideal" Red (0.1) and NIR (0.06) values for something that's perfectly burned.

*   If a pixel is **very burned**, its Red and NIR values will be very close to 0.1 and 0.06. This makes the denominator (the "distance") very small.
*   When you divide 1 by a very small number, you get a **LARGE BAI value**.
*   If a pixel is **not burned**, its Red and NIR values will be further away from 0.1 and 0.06. This makes the denominator larger.
*   When you divide 1 by a larger number, you get a **smaller BAI value**.

**🚦 CRUCIAL NOTE ON SCALING!**

Just like with EVI, the satellite data (especially from Landsat Collection 2 Surface Reflectance) often comes with its reflectance values multiplied by a big number (e.g., `0.0000275` and then an offset of `-0.2` is applied to get numbers like 5000 instead of 0.5).

The BAI formula **NEEDS** the `Red` and `NIR` values to be true surface reflectance, typically scaled between 0 and 1.

So, **BEFORE** you plug them into the BAI formula, you **MUST** apply the correct scaling factors provided by the satellite data provider (like USGS for Landsat). For Landsat 8/9 Collection 2 Level 2 data, this typically involves:
1.  Multiplying the original band value (e.g., `SR_B4`) by `0.0000275`.
2.  Adding `-0.2` to the result.

If you skip this scaling, your BAI values will be completely wrong! 😬

**What do BAI values mean?**
*   **Higher BAI values:** Stronger indication of burned area. These areas will often appear bright in a BAI image.
*   **Lower BAI values:** Indicate unburned vegetation, soil, water, or clouds.

BAI is a powerful tool for quickly mapping the extent of fire damage and for studying how landscapes recover over time.

In [3]:
# ---------------------------------------------------------------------------------
# Notebook: Burned Area Index (BAI) Calculation for a Selected Location
# ---------------------------------------------------------------------------------
# Purpose: Illustrate BAI calculation using Landsat 8 Surface Reflectance
#          for a user-selected location.
# Designed for IRAP program, University of Texas.
# Assumes 'ee', 'geemap', 'display' are imported and GEE is initialized.
# Assumes 'locations' dictionary is defined in a previous cell.
# ---------------------------------------------------------------------------------

# --- 1. Define Parameters for BAI Calculation ---

locations = {
    'rim_fire': { # November 2019
        'name': 'Rim Fire Area, California, USA',
        'coords': [37.850, -120.083]
    },
    'kincade_fire': {
        'name': 'Kincade Fire Area, Sonoma County, CA',
        'coords': [38.65, -122.70] # lat, lon
    },
    'dixie_fire_section': { #Octuber 2021
        'name': 'Dixie Fire (Section), Northern CA',
        'coords': [40.10, -121.20] # lat, lon
    }
}

# Landsat 8 Collection 2 Surface Reflectance Tier 1
# This is the atmospherically corrected data, good for analysis.
LANDSAT_COLLECTION_ID = 'LANDSAT/LC08/C02/T1_L2'

# Define a date range. For the Rim Fire, specific dates are relevant.
# You can adjust these if you select a different location/event.
# Example dates for Rim Fire (post-fire)
START_DATE = '2021-9-1'
END_DATE = '2021-10-30'

# Cloud cover threshold (e.g., less than 20%)
MAX_CLOUD_COVER = 20

# --- 2. Select a Location to Process ---
# MODIFY THIS LINE TO CHOOSE A DIFFERENT LOCATION FROM YOUR 'locations' DICTIONARY
# Or add a new entry to 'locations'
# Example: Add to your 'locations' dictionary:
# 'rim_fire': {
#     'name': 'Rim Fire Area, California, USA',
#     'coords': [37.850, -120.083]  # lat, lon
# }
selected_location_key = 'dixie_fire_section' # Make sure this key exists in your 'locations' dict
# -----------------------------------------

# Get the data for the selected location
if selected_location_key not in locations:
    raise ValueError(f"Error: The key '{selected_location_key}' is not defined in the 'locations' dictionary. "
                     f"Available keys are: {list(locations.keys())}")

selected_location_data = locations[selected_location_key]
location_name = selected_location_data['name']
# Coordinates are [latitude, longitude]
location_coords_lat_lon = selected_location_data['coords']
# Earth Engine's ee.Geometry.Point requires (longitude, latitude)
point_coords_lon_lat = [location_coords_lat_lon[1], location_coords_lat_lon[0]]

print(f"Processing BAI for location: {location_name} ({selected_location_key})")
print(f"Using date range: {START_DATE} to {END_DATE}")

# --- 3. Define Point of Interest (POI) for Earth Engine ---
poi = ee.Geometry.Point(point_coords_lon_lat) # lon, lat

# --- 4. Function to Apply Scaling Factors to Landsat Collection 2 SR ---
def apply_scale_factors(image):
    """
    Applies scaling factors to Landsat Collection 2 Level-2 (Surface Reflectance) data.
    Optical bands (SR_B*) are multiplied by 0.0000275 and offset by -0.2.
    Thermal bands (ST_B*) are multiplied by 0.00341802 and offset by 149.0.
    This function scales only the optical bands for BAI calculation.
    """
    # Select optical bands (those starting with 'SR_B')
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    # Select thermal bands (those starting with 'ST_B') and scale them
    # thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0) # Kelvin

    # Add the scaled bands back to the image, overwriting the original optical bands
    # and keeping other bands (like QA bands) if they exist.
    return image.addBands(optical_bands, None, True)
    # If you also need scaled thermal bands:
    # return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# --- 5. Retrieve and Process Landsat Imagery ---
# Load the Landsat image collection
image_collection = (ee.ImageCollection(LANDSAT_COLLECTION_ID)
                    .filterBounds(poi)
                    .filterDate(START_DATE, END_DATE)
                    .filter(ee.Filter.lt('CLOUD_COVER', MAX_CLOUD_COVER)))

# Apply the scaling factors to each image in the collection
scaled_collection = image_collection.map(apply_scale_factors)

# Check if any images were found after filtering
count = scaled_collection.size().getInfo()
if count == 0:
    raise Exception(f"No suitable Landsat images found for {location_name} "
                    f"between {START_DATE} and {END_DATE} with <{MAX_CLOUD_COVER}% cloud cover. "
                    "Try adjusting dates or cloud cover threshold.")

# Sort the collection by cloud cover (least cloudy first) and get the first image.
# Note: Sorting by 'CLOUD_COVER' uses the original metadata, which is fine.
selected_image = scaled_collection.sort('CLOUD_COVER').first()
print(f"Selected image ID: {selected_image.id().getInfo()}")

# --- 6. Calculate Burned Area Index (BAI) ---
# BAI is typically calculated as: 1 / ((0.1 - Red_scaled)^2 + (0.06 - NIR_scaled)^2)
# For Landsat 8:
#   Red band is 'SR_B4' (Surface Reflectance Band 4)
#   NIR band is 'SR_B5' (Surface Reflectance Band 5)
# Ensure you are using the scaled bands from `selected_image`.

# Select the scaled Red and NIR bands
red_band_scaled = selected_image.select('SR_B4')
nir_band_scaled = selected_image.select('SR_B5')

# Calculate BAI using an ee.Image.expression
bai_expression = selected_image.expression(
    '1.0 / ((0.1 - RED)**2 + (0.06 - NIR)**2)', {
        'RED': red_band_scaled,
        'NIR': nir_band_scaled
    }).rename('BAI')

# --- 7. Define Visualization Parameters ---
# For BAI: Higher values indicate a higher probability of burned area.
# The palette transitions from low probability (e.g., white/yellow) to high (e.g., red/black).
bai_vis_params = {
    'min': 0,
    'max': 350, # Adjust max based on typical BAI values for intense burns
    'palette': ['#FFFFFF', '#FFFF00', '#FFA500', '#FF0000', '#A52A2A', '#000000']
    # White, Yellow, Orange, Red, Brown, Black
}

# For Landsat 8 True Color (RGB) visualization using scaled Surface Reflectance bands
# Scaled bands are typically in the 0-1 range. A common stretch is 0-0.3.
rgb_vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'], # Red, Green, Blue bands
    'min': 0.0,
    'max': 0.3, # Adjust for brightness/contrast
    'gamma': 1.2 # Adjust gamma for contrast
}

# --- 8. Create and Display the Map using geemap ---
map_display_bai = geemap.Map(center=location_coords_lat_lon, zoom=10) # Adjust zoom as needed

# Add the Landsat True Color SR image (scaled)
map_display_bai.addLayer(
    selected_image,
    rgb_vis_params,
    f'Landsat True Color SR - {location_name}'
)

# Add the BAI layer
map_display_bai.addLayer(
    bai_expression,
    bai_vis_params,
    f'BAI (SR) - {location_name}'
)

# Add a marker at the point of interest
map_display_bai.add_marker(
    location=location_coords_lat_lon,
    name=location_name,
    draggable=False
)

# Add a layer control
map_display_bai.addLayerControl()

# Display the map
print(f"\nDisplaying map for {location_name}...")
display(map_display_bai)

print("\n--- BAI script execution finished ---")

Processing BAI for location: Dixie Fire (Section), Northern CA (dixie_fire_section)
Using date range: 2021-9-1 to 2021-10-30
Selected image ID: LC08_044032_20211015

Displaying map for Dixie Fire (Section), Northern CA...


Map(center=[40.1, -121.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…


--- BAI script execution finished ---
